In [31]:
# Importing core libraries required for the case study
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sea
import datetime as dt
import warnings

loan = pd.read_csv('./dataset/loan.csv', low_memory=False)

loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


# Data Cleaning and Manipulation
Aiming to clean and manipulate the data to finally reach a data form where we can work with the data. Will wilcude the following steps while cleaning and manipulating data.
1. Find the number of columns which do not have any data.
2. Drop unwanted and duplicate rows.
3.


## Getting the summary of all columns which contain null values
Print summary of Nulls, Blanks in the dataset the ine form a percentage value.

In [32]:
(loan.isnull().sum() / len(loan.index) * 100)

id                              0.000000
member_id                       0.000000
loan_amnt                       0.000000
funded_amnt                     0.000000
funded_amnt_inv                 0.000000
                                 ...    
tax_liens                       0.098195
tot_hi_cred_lim               100.000000
total_bal_ex_mort             100.000000
total_bc_limit                100.000000
total_il_high_credit_limit    100.000000
Length: 111, dtype: float64

## Removing All Rows With loan_status = "Current"
Loan Status - Key Leading Attribute (loan_status). The column has three distinct values
- Fully-Paid - The customer has successfuly paid the loan
- Charged-Off - The customer is "Charged-Off" ir has "Defaulted"
- Current - These customers, the loan is currently in progress and cannot contribute to conclusive evidence if the customer will default of pay in future
**For the given case study, "Current" status rows will be ignored**

In [38]:
prev_rows = len(loan)
# The rows where loan_stats=Current are the data where the loan repayment is currently in progress
# The loans which are currently in progress will not contribute to decisions
# of default or pass as it's difficult to predict the outcome
#
# Dropping the rows early as, dropping all Current rows introduces NA columns which can be easily dropped
loan = loan[loan['loan_status'] != "Current"]

# Print current data statistics after dropping rows with loan_status "CURRENT"
curr_rows = len(loan)
print(curr_rows)
print(prev_rows)

print("Number of rows dropped where loan_status = 'Current':", (prev_rows - curr_rows))
print("Percentage of rows dropped = ", round((prev_rows - curr_rows) / prev_rows * 100, 2), "%")

38577
38577
Number of rows dropped where loan_status = 'Current': 0
Percentage of rows dropped =  0.0 %


## Find any duplicate rows in the dataset

In [35]:
duplicate_rows = len(loan[loan.duplicated()])
if duplicate_rows <= 0:
    print("Duplicate Rows: ", duplicate_rows)
    print("No action needed")
else:
    print(len(duplicate_rows))

Duplicate Rows:  0
No action needed


### Dropping Columns That Have ID Values
Dropping columns which is unique id in nature. They dont contribute to loan analysis

In [39]:
# Checking if member_id is unique
if len(loan['member_id'].unique()) == loan.shape[0]:
    print("member_id is unique, can be dropped")
    loan = loan.drop(['member_id'], axis=1)
else:
    print("member_id is not unique, dont drop")

# Checking if id is unique
if len(loan['id'].unique()) == loan.shape[0]:
    print("id is unique, can be dropped")
    # not dropping id as it will be used for pivot calculation later
    # loan = loan.drop(['id'],  axis=1)
else:
    print("id is not unique, dont drop")

member_id is unique, can be dropped
id is unique, can be dropped


### Dropping text/description columns which will not contribute to overall analysis
These are names of establishment etc which will not contribute to loan pass or failure. The URL column is a static link with id as the attribute. It's a redundant column

In [ ]:
loan = loan.drop(['url', 'emp_title', 'desc', 'title'], axis=1)

# Dropping column sub_grade as the current analysis will limit to Grade only

In [40]:
loan = loan.drop(['sub_grade'], axis=1)

### Dropping all columns which refer to behavioural data of customer post loan approval
Behaviour data of the customers are captured post the loan approval. The data is not available at the time of loan approval and thus cannot be used for calculations

In [41]:
loan = loan.drop(['delinq_2yrs', 'earliest_cr_line',
                  'inq_last_6mths', 'open_acc', 'pub_rec',
                  'revol_bal', 'revol_util', 'total_acc',
                  'out_prncp', 'out_prncp_inv', 'total_pymnt',
                  'total_pymnt_inv', 'total_rec_prncp',
                  'total_rec_int', 'total_rec_late_fee', 'recoveries',
                  'collection_recovery_fee', 'last_pymnt_d',
                  'last_pymnt_amnt', 'last_credit_pull_d',
                  'application_type'], axis=1)

### Dropping all columns whose all the values are NA
Print all NA columns for verification

In [42]:
print("Total columns with values NA: ", len(loan.columns[loan.isna().all()].tolist()),"\n\n")
print("Columns with all values as NA\n", loan.columns[loan.isna().all()].tolist())

# Dropping all the columns whose all the records are NaN or Null
loan = loan.dropna(axis='columns', how="all")

Total columns with values NA:  55 


Columns with all values as NA
 ['next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_7

### Dropping all columns with all zero values

In [ ]:
loan = loan.loc[:, (loan != 0).any(axis=0)]

### Function to Drop all columns who have constant values (ignoring NA value)
- Example most of the columns is 1 and rest is NA, the column will be dropped
- If we have 1,2 and NA, the column wont be dropped

In [47]:
print("Columns with constant values with or without NA")
def drop_constant_columns(df):
    for c in df.columns:
        if df[c].nunique(dropna=True) == 1:
            print(c)
            df = df.drop(c, axis=1)
    return df

# Drop all constant columns from df1 (definition of constant is constant value across the rows, this ignores Na values)
loan = drop_constant_columns(loan)

Columns with constant values with or without NA


### Drop Columns With More Than 65% Empty Values
Function `drop_mostly_empty_columns` which checks the amount of empty values in a dataframe and drops the column if the amount of empty values is more than 65%. 60% is the threshhold percentage which decides imputing vs dropping.

In [48]:
print("Columns with more that 65% empty records")
def drop_mostly_empty_columns(df):
    total_rows = len(df)
    for c in df.columns:
        # Drop columns whose mean na values exceed 65%
        if df[c].isna().mean().round(2) >= 0.65:
            print(c)
            df = df.drop(c, axis=1)
    return df
loan = drop_mostly_empty_columns(loan)

Columns with more that 65% empty records


# Data Conversion
Since we have removed unwanted data from the dataset now we can focus on data conversion. During this process we will focus on the following:
1. Converting columns to floats where needed.
2. Converting required columns to integers if they can be converted and exist as strings in dataset.
3. Round off any values in the columns which have floating point values with more than 2 decimal places.

#### Convert the columns loan_amnt and funded_amnt as float64

In [49]:
loan = loan.astype({'loan_amnt':'float','funded_amnt':'float'})

#### Convert the term column into an integer from a string

In [50]:
loan['term'] = loan['term'].apply(lambda x : int(x[:-7]))

#### Convert int_rate to  float by removing the "%" character

In [51]:
loan['int_rate'] = loan['int_rate'].apply(lambda x : float(x[:-1]))

#### Round off the values of key float fields to 2 decimal place

In [52]:
print("Rounding columns to 2 decimal places")
for c in ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'dti']:
    print(c)
    loan[c] = loan[c].apply(lambda x: round(x,2))

Rounding columns to 2 decimal places
loan_amnt
funded_amnt
funded_amnt_inv
int_rate
dti


#### Converting the loan_status to boolean column.
"Fully-Paid is True and Charged Off is False"
- Added a function instead of lambda because, if this is accidentally re-run on a boolean column, the logic broke
- Now it will only convert to boolean if the column is a string and has the two specific values

In [53]:
def convert_loan_status_to_boolean(x):
    if x == "Fully Paid":
        return True
    elif x == "Charged Off":
        return False
    else:
        return x

In [54]:
# Converting the column issue_d from string object to DateTime
loan['issue_d'] = pd.to_datetime(loan['issue_d'], format='%b-%y')

### Deciding to Impute Or Drop Columns
Evaluating the percentage of rows with blank values for both the columns. If the total percentage is less than 5% will take an option of dropping the columns vs imputing.

In [55]:
# Identify columns who have blank values and what percentage of total values are there blanks.
# These values may need to be imputed
print('emp_length blank rows', loan['emp_length'].isna().sum())
print('pub_rec_bankruptcies blank rows', loan['pub_rec_bankruptcies'].isna().sum(), "\n")
imp_total = 0
for c in loan.columns[loan.isna().any()].tolist():
    imp_val = len(loan[loan[c].isna()]) / len(loan) * 100
    imp_total += imp_val
    print(c, round(imp_val, 2),"%")
print("\nTotal rows percentage whose columns are blank: ", round(imp_total ,2), "%")

emp_length blank rows 1033
pub_rec_bankruptcies blank rows 697 

emp_title 6.19 %
emp_length 2.68 %
desc 32.47 %
title 0.03 %
pub_rec_bankruptcies 1.81 %

Total rows percentage whose columns are blank:  43.17 %


#### Converting emp_length to integer values
Converting emp_length as numerical data to create more effective statistical analysis as compared to nominal values


In [56]:
loan['emp_length'] = loan['emp_length'].replace({'< 1 year': 0, '2 years': 2, '3 years': 3,
                                                 '7 years': 7, '4 years': 4, '5 years': 5,
                                                 '1 year': 1, '6 years': 6, '8 years': 8,
                                                 '9 years': 9,  '10+ years': 10})

In [ ]:
# Save the current dimensions of the dataframe in a variable rows_before
rows_before = len(loan)

# Drop rows with empty values in this scenario
Since the percent of rows is very small, dropping the rows instead of imputing them

In [58]:
loan = loan[loan['emp_length'].notna()]
loan = loan[loan['pub_rec_bankruptcies'].notna()]

# Checking if blanks exist
loan['emp_length'].value_counts()

10.0    8369
0.0     4341
2.0     4207
3.0     3951
4.0     3297
5.0     3161
1.0     3077
6.0     2136
7.0     1689
8.0     1410
9.0     1209
Name: emp_length, dtype: int64

In [59]:
# Print the dimensions of the dataframe after dropping rows
rows_after = len(loan)
print("Number of rows dropped = ,", (rows_before - rows_after))
print("Percentage of rows dropped = ", round((rows_before - rows_after)/rows_before*100,2),"%")
print(loan.shape)

Number of rows dropped = , 1730
Percentage of rows dropped =  4.48 %
(36847, 23)


## Deriving Additional Columns
Adding additional column for Year and Month for analysis extrating Year and Month from issue_d